# Imports

In [40]:
import cv2
import numpy as np
#from skimage import data, util
#from skimage.measure import label, regionprops

# Using MSER


# Getting the Image

In [41]:
#MSER will detect clusters of data where in this case, individual words = clusters of data.

#MSER object
mser = cv2.MSER_create()

#The image to read
img = cv2.imread('test5.jpg') 
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) #gray scale the image

vis = img.copy()



# Detecting Regions (Words)

#                      Find Regions

coordinates, bboxes = mser.detectRegions(img_gray) #words in the gray scale image

coords = []
for coord in coordinates:
    bbox = cv2.boundingRect(coord)
    x,y,w,h = bbox
    if w < 20 or h < 20 or w/h > 10 or h/w > 10:
        continue
    coords.append(coord)
    
colors = [[43, 43, 200], [43, 75, 200], [43, 106, 200], [43, 137, 200], [43, 169, 200], [43, 200, 195], [43, 200, 163], [43, 200, 132], [43, 200, 101], [43, 200, 69], [54, 200, 43], [85, 200, 43], [116, 200, 43], [148, 200, 43], [179, 200, 43], [200, 184, 43], [200, 153, 43], [200, 122, 43], [200, 90, 43], [200, 59, 43], [200, 43, 64], [200, 43, 95], [200, 43, 127], [200, 43, 158], [200, 43, 190], [174, 43, 200], [142, 43, 200], [111, 43, 200], [80, 43, 200], [43, 43, 200]]

np.random.seed(0)
canvas1 = img.copy()
canvas2 = cv2.cvtColor(img_gray, cv2.COLOR_GRAY2BGR)
canvas3 = np.zeros_like(img)

for cnt in coords:
    xx = cnt[:,0]
    yy = cnt[:,1]
    color = colors[np.random.choice(len(colors))]
    canvas1[yy, xx] = color
    canvas2[yy, xx] = color
    canvas3[yy, xx] = color
    
cv2.imshow('img', canvas1)
cv2.waitKey(0)

# Define Region Properties to Reduce Noise Detection

mserStats = regionprops(bboxes) #properties of each bbox as a tuple of regions
bboxStats = []
bboxStats.append(mserStats[0].bbox)
print(bboxes)
for i in range(1, len(mserStats)):
    myInput = []
    myInput.append(mserStats[i].bbox)
    bboxStats = np.concatenate((bboxStats, myInput))

w = bboxStats[:, 2]
h = bboxStats[:, 3]
aspectRatio = w/h
print(len(bboxes))
filterIdx = aspectRatio.conj().transpose() > 30
#print(filterIdx)

for i in range(0, len(mserStats)):
    #print(mserStats[i].eccentricity)
    filterIdx[i] = filterIdx[i] or (mserStats[i].eccentricity > 1)
    filterIdx[i] = filterIdx[i] or (mserStats[i].solidity < .1)
    filterIdx[i] = filterIdx[i] or ((mserStats[i].extent < 0.2) | (mserStats[i].extent > 0.9))
    filterIdx[i] = filterIdx[i] or (mserStats[i].euler_number < -4)

print(filterIdx)
#mserStats[filterIdx] = []
#mserRegions[filterIdx] = []
for j in range(0, len(mserStats)):
    if(filterIdx[j] == True):
        mserStats[j] = []
        #print(j)
        np.delete(bboxes, [0])

print(len(bboxes))
    

In [42]:
_, bboxes = mser.detectRegions(img_gray)
bboxes_list = []
heights = []

for bbox in bboxes:
    x, y, w, h = bbox
    bboxes_list.append([x, y, x + w, y + h])
    heights.append(h)
    cv2.rectangle(vis, (x, y), ((x+h), (y+w)), (0, 255, 0), 2)
print(len(bboxes))
heights = sorted(heights)
median_height = heights[int(len(heights) / 2)] / 2

#hulls = [cv2.convexHull(p.reshape(-1, 1, 2)) for p in regions]
#cv2.polylines(vis, hulls, 1, (0, 255, 0))
cv2.imshow('img', vis)
cv2.waitKey(0)

#mask = np.zeros((img.shape[0], img.shape[1], 1), dtype = np.uint8)
#for contour in hulls:
    #cv2.drawContours(mask, [contour], -1, (255, 255, 255), -1)
#text_only = cv2.bitwise_and(img, img, mask = mask)
#cv2.imshow("text only", text_only)
#cv2.waitKey(0)



163


-1

In [38]:
def grouper(iterable, interval=0):
    prev = None
    group = []
    for item in iterable:
        if not prev or abs(item[1] - prev[1]) <= interval:
            group.append(item)
        else:
            yield group
            group = [item]
        prev = item
    if group:
        yield group

def non_max_suppression_fast(boxes, overlapThresh):
   # if there are no boxes, return an empty list
   if len(boxes) == 0:
      return []

   # if the bounding boxes integers, convert them to floats --
   # this is important since we'll be doing a bunch of divisions
   if boxes.dtype.kind == "i":
      boxes = boxes.astype("float")
#  
   # initialize the list of picked indexes   
   pick = []

   # grab the coordinates of the bounding boxes
   x1 = boxes[:,0]
   y1 = boxes[:,1]
   x2 = boxes[:,2]
   y2 = boxes[:,3]
    # compute the area of the bounding boxes and sort the bounding
   # boxes by the bottom-right y-coordinate of the bounding box
   area = (x2 - x1 + 1) * (y2 - y1 + 1)
   idxs = np.argsort(y2)

   # keep looping while some indexes still remain in the indexes
   # list
   while len(idxs) > 0:
      # grab the last index in the indexes list and add the
      # index value to the list of picked indexes
      last = len(idxs) - 1
      i = idxs[last]
      pick.append(i)

      # find the largest (x, y) coordinates for the start of
      # the bounding box and the smallest (x, y) coordinates
      # for the end of the bounding box
      xx1 = np.maximum(x1[i], x1[idxs[:last]])
      yy1 = np.maximum(y1[i], y1[idxs[:last]])
      xx2 = np.minimum(x2[i], x2[idxs[:last]])
      yy2 = np.minimum(y2[i], y2[idxs[:last]])
      
      # compute the width and height of the bounding box
      w = np.maximum(0, xx2 - xx1 + 1)
      h = np.maximum(0, yy2 - yy1 + 1)

      # compute the ratio of overlap
      overlap = (w * h) / area[idxs[:last]]

      # delete all indexes from the index list that have
      idxs = np.delete(idxs, np.concatenate(([last],
         np.where(overlap > overlapThresh)[0])))

   # return only the bounding boxes that were picked using the
   # integer data type
   return boxes[pick].astype("int")

bboxes_list = non_max_suppression_fast(bboxes_list, .3)  # Group the bounding boxes
for (startX, startY, endX, endY) in bboxes:
    cv2.rectangle(vis, (startX, startY), (endX, endY), (0, 255, 0), 2)
    
print(len(bboxes))


cv2.imshow('img', img)
cv2.waitKey(0)

AttributeError: 'list' object has no attribute 'dtype'